# Transfer Learning CIFAR10

* Train a simple convnet on the CIFAR dataset the first 5 output classes [0..4].
* Freeze convolutional layers and fine-tune dense layers for the last 5 ouput classes [5..9].


Ignoring warnings

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import warnings
warnings.filterwarnings('ignore')

Importing Libraries

In [2]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

### 1. Import CIFAR10 data and create 2 datasets with one dataset having classes from 0 to 4 and other having classes from 5 to 9 

Import CIFAR10 Data

In [3]:
import keras
from keras.datasets import cifar10
from keras.layers import Dense, Convolution2D, Flatten, Activation, MaxPooling2D, Dropout
from keras.models import Sequential
from keras.utils import np_utils


Using TensorFlow backend.


Train and Test the dataset

In [4]:
(X_Train, Y_Train), (X_Test, Y_Test) = cifar10.load_data()
Examples = 50000


X1_Train = []
X1_Test = []
X2_Train = []
X2_Test = []
Y1_Train = []
Y1_Test = []
Y2_Train = []
Y2_Test = []

Put the data in the datasets

In [5]:
for ix in range(Examples):
    if Y_Train[ix] < 5:
        X1_Train.append(X_Train[ix]/255.0)
        Y1_Train.append(Y_Train[ix])
    else:
        X2_Train.append(X_Train[ix]/255.0)
        Y2_Train.append(Y_Train[ix])

for ix in range(Y_Test.shape[0]):
    if Y_Test[ix] < 5:
        X1_Test.append(X_Test[ix]/255.0)
        Y1_Test.append(Y_Test[ix])
    else:
        X2_Test.append(X_Test[ix]/255.0)
        Y2_Test.append(Y_Test[ix])


In [6]:
len(X1_Train)

25000

### Reshape the test and train datasets

In [7]:
X1_Train = np.asarray(X1_Train).reshape((-1, 32, 32, 3))
X1_Test = np.asarray(X1_Test).reshape((-1, 32, 32, 3))
X2_Train = np.asarray(X2_Train).reshape((-1, 32, 32, 3))
X2_Test = np.asarray(X2_Test).reshape((-1, 32, 32, 3))

print(X1_Train.shape, X1_Test.shape)


(25000, 32, 32, 3) (5000, 32, 32, 3)


### 2. Use One-hot encoding to divide y_train and y_test into required no of output classes

### Creating y_train adn Y_test

In [9]:
Y1_Train = np_utils.to_categorical(np.asarray(Y1_Train), num_classes=5)
Y1_Test = np_utils.to_categorical(np.asarray(Y1_Test), num_classes=5)

Y2_Train = np_utils.to_categorical(np.asarray(Y2_Train), num_classes=10)
Y2_Test = np_utils.to_categorical(np.asarray(Y2_Test), num_classes=10)
print(Y1_Train.shape, Y1_Test.shape)

(25000, 5, 5) (5000, 5, 5)


In [10]:
#Put the data

In [11]:
set1 = int(0.8 * X1_Train.shape[0])
set2 = int(0.8 * X2_Train.shape[0])

X1_Vals = X1_Train[set1:]
X1_train = X1_Train[:set1]
Y1_Vals = Y1_Train[set1:]
Y1_train = Y1_Train[:set1]

X2_Vals = X2_Train[set2:]
X2_train = X2_Train[:set2]
Y2_Vals = Y2_Train[set2:]
Y2_Train = Y2_Train[:set2]

### 3. Build a sequential neural network model which can classify the classes 0 to 4 of CIFAR10 dataset with at least 80% accuracy on test data

In [12]:
Model1 = Sequential()

Model1.add(Convolution2D(32, 5, 5, input_shape=(32, 32, 3), activation='relu'))
Model1.add(Convolution2D(16, 5, 5, activation='relu'))
Model1.add(MaxPooling2D(pool_size=(2,2)))
Model1.add(Convolution2D(8, 3, 3, activation='relu'))
Model1.add(Flatten())
Model1.add(Dropout(0.42))

Model1.add(Dense(128))
Model1.add(Activation('relu'))

Model1.add(Dense(5))
Model1.add(Activation('softmax'))

Model1.summary()
Model1.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 28, 28, 32)        2432      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 16)        12816     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 16)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 10, 10, 8)         1160      
_________________________________________________________________
flatten_1 (Flatten)          (None, 800)               0         
_________________________________________________________________
dropout_1 (Dropout)  

### Testing

In [9]:
import time, datetime

starttime = datetime.datetime.now()
test = Model1.fit(x1_train, y1_train,
         nb_epoch=25,
         shuffle=True,
         batch_size=100,
         validation_data=(x1_val, y1_val), verbose=2)

time_taken = datetime.datetime.now() - starttime
print('\n'*2, '-'*20, '\n')
print('Time taken for first training: ', time_taken)
print('\n', '-'*20, '\n'*2)


Instructions for updating:
Use tf.cast instead.
Train on 20000 samples, validate on 5000 samples
Epoch 1/25
 - 76s - loss: 1.2346 - acc: 0.4748 - val_loss: 1.0432 - val_acc: 0.5790
Epoch 2/25
 - 76s - loss: 1.0111 - acc: 0.5801 - val_loss: 0.9453 - val_acc: 0.6236
Epoch 3/25
 - 76s - loss: 0.9423 - acc: 0.6153 - val_loss: 0.9138 - val_acc: 0.6328
Epoch 4/25
 - 77s - loss: 0.8987 - acc: 0.6357 - val_loss: 0.8630 - val_acc: 0.6616
Epoch 5/25
 - 76s - loss: 0.8618 - acc: 0.6530 - val_loss: 0.8378 - val_acc: 0.6684
Epoch 6/25
 - 78s - loss: 0.8243 - acc: 0.6735 - val_loss: 0.8408 - val_acc: 0.6736
Epoch 7/25
 - 76s - loss: 0.7985 - acc: 0.6844 - val_loss: 0.8031 - val_acc: 0.6926
Epoch 8/25
 - 83s - loss: 0.7753 - acc: 0.6936 - val_loss: 0.8089 - val_acc: 0.6888
Epoch 9/25
 - 77s - loss: 0.7476 - acc: 0.7077 - val_loss: 0.8019 - val_acc: 0.6906
Epoch 10/25
 - 78s - loss: 0.7222 - acc: 0.7189 - val_loss: 0.7728 - val_acc: 0.7026
Epoch 11/25
 - 76s - loss: 0.7139 - acc: 0.7193 - val_loss: 0.

## We can see that it Achieved 80% accuracy in 25th epoch and a validation accuracy is 72.86%. However, the training time was 31 minutes and 34 seconds.


### 4. In the model which was built above (for classification of classes 0-4 in CIFAR10), make only the dense layers to be trainable and conv layers to be non-trainable

In [10]:
for l in model.layers[:6]:
    l.trainable = False

### 5. Utilize the the model trained on CIFAR 10 (classes 0 to 4) to classify the classes 5 to 9 of CIFAR 10  (Use Transfer Learning) <br>
Achieve an accuracy of more than 85% on test data

In [11]:
re_model = Sequential(model.layers[:6])

re_model.add(Dense(128))
re_model.add(Activation('relu'))
re_model.add(Dense(10))
re_model.add(Activation('softmax'))

re_model.summary()
re_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 28, 28, 32)        2432      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 16)        12816     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 16)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 10, 10, 8)         1160      
_________________________________________________________________
flatten_1 (Flatten)          (None, 800)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 800)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 128)               102528    
__________

In [12]:
starttime = datetime.datetime.now()
test2 = trans_model.fit(X2_train, Y2_train, nb_epoch=25, shuffle=True, batch_size=100, validation_data=(X2_Vals, Y2_Vals), verbose=2)
time_taken = datetime.datetime.now() - starttime
print('\n'*2, '-'*20, '\n')
print('Time taken for first training: ', time_taken)
print('\n', '-'*20, '\n'*2)

Train on 20000 samples, validate on 5000 samples
Epoch 1/25
 - 35s - loss: 0.8946 - acc: 0.6637 - val_loss: 0.7356 - val_acc: 0.7290
Epoch 2/25
 - 35s - loss: 0.7633 - acc: 0.7114 - val_loss: 0.7372 - val_acc: 0.7318
Epoch 3/25
 - 35s - loss: 0.7375 - acc: 0.7222 - val_loss: 0.6987 - val_acc: 0.7486
Epoch 4/25
 - 35s - loss: 0.7051 - acc: 0.7328 - val_loss: 0.6828 - val_acc: 0.7554
Epoch 5/25
 - 35s - loss: 0.6777 - acc: 0.7416 - val_loss: 0.6684 - val_acc: 0.7556
Epoch 6/25
 - 35s - loss: 0.6550 - acc: 0.7527 - val_loss: 0.6630 - val_acc: 0.7570
Epoch 7/25
 - 35s - loss: 0.6331 - acc: 0.7619 - val_loss: 0.6449 - val_acc: 0.7686
Epoch 8/25
 - 35s - loss: 0.6122 - acc: 0.7667 - val_loss: 0.6434 - val_acc: 0.7688
Epoch 9/25
 - 36s - loss: 0.5946 - acc: 0.7733 - val_loss: 0.6324 - val_acc: 0.7756
Epoch 10/25
 - 35s - loss: 0.5747 - acc: 0.7838 - val_loss: 0.6323 - val_acc: 0.7752
Epoch 11/25
 - 35s - loss: 0.5593 - acc: 0.7897 - val_loss: 0.6299 - val_acc: 0.7740
Epoch 12/25
 - 35s - loss

## We can see that it achieved 85% accuracy using 25th Epoch. The second model trained for classes 5..9 used transfer learning.  It achieved validadation accuracy of 77.50% using 25 epochs. Train time was down to just 14 minnutes and 43 seconds.



## Therefore, transfer learning significantly brought down the time for processing of data.

# Text classification using TF-IDF

### 6. Load the dataset from sklearn.datasets

In [13]:
from sklearn.datasets import fetch_20newsgroups

In [14]:
categories = ['alt.atheism', 'soc.religion.christian', 'comp.graphics', 'sci.med']

### 7. Training data

In [15]:
twenty_train = fetch_20newsgroups(subset='train', categories=categories, shuffle=True, random_state=42)

In [16]:
type(twenty_train)

sklearn.utils.Bunch

In [17]:
len(twenty_train)

6

### 8. Test data

In [18]:
twenty_test = fetch_20newsgroups(subset='test', categories=categories, shuffle=True, random_state=42)

In [19]:
len(twenty_test)

6

###  a.  You can access the values for the target variable using .target attribute 
###  b. You can access the name of the class in the target variable with .target_names


In [20]:
twenty_train.target

array([1, 1, 3, ..., 2, 2, 2], dtype=int64)

In [21]:
twenty_train.target_names

['alt.atheism', 'comp.graphics', 'sci.med', 'soc.religion.christian']

In [22]:
twenty_train.data[0:5]

['From: sd345@city.ac.uk (Michael Collier)\nSubject: Converting images to HP LaserJet III?\nNntp-Posting-Host: hampton\nOrganization: The City University\nLines: 14\n\nDoes anyone know of a good way (standard PC application/PD utility) to\nconvert tif/img/tga files into LaserJet III format.  We would also like to\ndo the same, converting to HPGL (HP plotter) files.\n\nPlease email any response.\n\nIs this the correct group?\n\nThanks in advance.  Michael.\n-- \nMichael Collier (Programmer)                 The Computer Unit,\nEmail: M.P.Collier@uk.ac.city                The City University,\nTel: 071 477-8000 x3769                      London,\nFax: 071 477-8565                            EC1V 0HB.\n',
 "From: ani@ms.uky.edu (Aniruddha B. Deglurkar)\nSubject: help: Splitting a trimming region along a mesh \nOrganization: University Of Kentucky, Dept. of Math Sciences\nLines: 28\n\n\n\n\tHi,\n\n\tI have a problem, I hope some of the 'gurus' can help me solve.\n\n\tBackground of the probl

In [ ]:
#Xtrain

In [23]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_Train_counts = count_vect.fit_transform(twenty_train.data)
X_Train_counts.shape

(2257, 35788)

### 9.  Now with dependent and independent data available for both train and test datasets, using TfidfVectorizer fit and transform the training data and test data and get the tfidf features for both

In [13]:
# Import libraries NLP

In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [29]:
vectorizer = TfidfVectorizer(min_df=0.01, max_df=0.95)
train = vectorizer.fit_transform(twenty_train.data)
test = vectorizer.transform(twenty_test.data)
train = train.todense()
test = test.todense()
train_labels = twenty_train.target
test_labels = twenty_test.target

print(train.shape)
print(test.shape)

print(train, train_labels, test, test_labels)

(2257, 2384)
(1502, 2384)
[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.01186874 0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]] [1 1 3 ... 2 2 2] [[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.06224145 ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]] [2 2 2 ... 2 2 1]


In [ ]:
#Train the datasets using count vectorizer
cvec = CountVectorizer()
train_features=cvec.fit(twenty_train.data)
cvecdata= cvec.transform(twenty_train.data)
df=pd.DataFrame(cvecdata.todense(),columns=cvec.get_feature_names())

In [30]:
print("Features : ", df)

Features :        00  000  0000  0000001200  000005102000  0001  000100255pixel  00014  \
0      0    0     0           0             0     0               0      0   
1      0    0     0           0             0     0               0      0   
2      0    0     0           0             0     0               0      0   
3      0    0     0           0             0     0               0      0   
4      1    0     0           0             0     0               0      0   
5      1    0     0           0             0     0               0      0   
6      0    0     0           0             0     0               0      0   
7      0    0     0           0             0     0               0      0   
8      0    0     0           0             0     0               0      0   
9      0    0     0           0             0     0               0      0   
10     0    0     0           0             0     0               0      0   
11     0    0     0           0             0     0 

### 10. Use logisticRegression with tfidf features as input and targets as output and train the model and report the train and test accuracy score

In [31]:
from sklearn.linear_model import LogisticRegression
y=twenty_train.target
model=LogisticRegression()
log_model=model.fit(df,y)

In [32]:
X_Test=cvec.transform(twenty_test.data)
Y_Test=twenty_test.target
log_model.score(X_Test,Y_Test)

0.9007989347536618

In [34]:
test_pred = model.predict(X_Test)
from sklearn import metrics
print(metrics.classification_report(Y_Test, test_pred))
print(metrics.confusion_matrix(Y_Test, test_pred))

             precision    recall  f1-score   support

          0       0.93      0.81      0.86       319
          1       0.88      0.96      0.92       389
          2       0.93      0.84      0.88       396
          3       0.88      0.98      0.93       398

avg / total       0.90      0.90      0.90      1502

[[258   8  13  40]
 [  5 373   9   2]
 [ 12  40 333  11]
 [  3   3   3 389]]


## It achieved 90% accuracy